## 4 дз питон


#### Парсим JSON файл, заводим train_log , парсим входые данные и название встроенной модели, которую будем обучать, оубчаем модель, сохраняем ее в папке model_dir

In [ ]:
@app.route("/fit", methods=['POST'])
def fit():
    input_json = request.get_json()
    config = input_json['config']
    model_name = config["model_name"]
    model_type = config['model_type']

    X = input_json['X']
    y = input_json['y']
    
    train_log = ""
    #train
    if model_type == "CatboostRegressor" :
        train_log += "CatboostRegressor fitting :\n"
        X = np.array(X)
        y = np.array(y)
        train_data = Pool(data=X, label=y)
        model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6)
        model.fit(train_data)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model.save_model(os.path.join(model_dir, model_name))
        train_log += "Success"
    elif model_type == "sklearn.linear_model":
        train_log += "\nSklearn linear model fitting :\n"
        X = np.array(X)
        y = np.array(y)
        model = LinearRegression()
        model.fit(X, y)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        joblib.dump(model, model_dir + '/' + model_name )
        train_log += "Success"
    else :
        return {'message': 'this type model dont exists' , "train_log" : train_log}

    return {"message": f"Model '{model_name}' trained and saved" ,
            "train_log" : train_log
            }

#### Парсим тестовые данные, название модели, ее тип, находим ее в папке, запускаем на тестовой выборке, выводим рещультат предсказания

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    input_json = request.get_json()
    X_test = input_json['X_test']
    config = input_json['config']
    model_type = config['model_type']
    model_name = config["model_name"]
    if model_type == "CatboostRegressor" :
        loaded_model = CatBoostRegressor()
        loaded_model.load_model(model_dir + '/' + model_name )
        result = loaded_model.predict(X_test)
    elif model_type == "sklearn.linear_model":
        loaded_model = LinearRegression()
        loaded_model.load_model(model_dir + '/' + model_name )
        result = loaded_model.predict(X_test)
    else :
        return {'message': 'this type model dont exists' }
    return {'predict' : list(result)}

#### Парсим название модели , и ее тип, парсим саму модель в декодированном виде, обратно ее кодируем и сохраняем в папке

In [ ]:
@app.post("/load")
def load():
    input_json = request.get_json()
    config = input_json['config']
    model_type = config['model_type']
    model_name = config["model_name"]
    decoded_model = input_json['decoded_model']
    model_path = os.path.join(model_dir, f"{model_name}")
    if os.path.exists(model_path):
        return {"error": f"Model '{model_name}' already exists"}
    if model_type == "CatboostRegressor" :
        model = pickle.loads(decoded_model.encode('latin1'))
        model.save_model(os.path.join(model_dir, model_name))
    elif model_type == "sklearn.linear_model":
        model = pickle.loads(decoded_model.encode(latin1))
        joblib.dump(model, model_dir + '/' + model_name )
    else :
        return {'message': 'this type model dont exists' }
    return {"message": f"Model '{model_name}' loaded for inference"}

#### парсим имена модели и ее тип, дальше выгружаем ее из папки, декодируем, отправляем, позже клиент ее кодирует и запускает

In [ ]:
@app.post("/unload")
def unload():
    input_json = request.get_json()
    config = input_json['config']
    model_type = config['model_type']
    model_name = config["model_name"]
    model_path = os.path.join(model_dir, f"{model_name}")
    
    if model_type == "CatboostRegressor" :
        model = CatBoostRegressor()
        model.load_model(model_path)
        decoded_model = pickle.dumps(model).decode("latin1")
        result = decoded_model   
    elif model_type == "sklearn.linear_model":
        model = LinearRegression()
        model.load_model(model_path)
        decoded_model = pickle.dumps(model).decode("latin1")
        result = decoded_model
    else :
        return {'message': 'this type model dont exists' }
    return {"decoded_model": result , "message": f"Model '{model_name}' unloaded"}

#### По названию ищем модель в папке и удаляем / просто все в папке с моделями удалем

In [ ]:
@app.post("/remove")
def remove():
    input_json = request.get_json()
    config = input_json['config']
    model_name = config["model_name"]
    model_path = os.path.join(model_dir, f"{model_name}")
    os.remove(model_path)
    return {"message": f"Model '{model_name}' removed"}

@app.post("/remove_all")
def remove_all():
    shutil.rmtree(model_dir)
    os.makedirs(model_dir)
    return {"message": "All models removed"}